In [70]:
import numpy as np
import pandas as pd
import cv2
import os
import time
# import keras.layers as kr
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle

from keras.models import Model
from keras.applications.resnet import *
# from keras.preprocessing.image_dataset import *
from keras.preprocessing import image
import random

In [52]:
train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values

In [55]:
random.seed(10)

all_locations_name = list(train_xy['id'])
all_locations_name_step5 = [all_locations_name[x:x+5] for x in range(0, len(all_locations), 5)]
validate_locations_name = random.sample(all_locations_step5 ,1200)
validate_name = [random.choice(validate_location) for validate_location in validate_locations_name]
train_name = [train for train in all_locations if train not in validate_name]

In [77]:
out_validate=[]
for i in range(len(validate_name)):
    coor = train_xy[train_xy['id'] == validate_name[i]]
    out_validate.append([validate_name[i], coor.iloc[0]['x'], coor.iloc[0]['y']])
    
validate_csv = pd.DataFrame(out_validate, columns=['id','x','y'])
validate_csv.to_csv('validate.csv',index=False)

In [81]:
out_train=[]
for i in range(len(train_name)):
    coor = train_xy[train_xy['id'] == train_name[i]]
    out_train.append([train_name[i], coor.iloc[0]['x'], coor.iloc[0]['y']])
    
train_csv = pd.DataFrame(out_train, columns=['id','x','y'])
train_csv.to_csv('validate_train.csv',index=False)

In [60]:
validate_i=[]
train_i=[]
for validate in validate_name:
    img = image.load_img('./train/' + validate + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    validate_i.append(x)
    
for train in train_name:
    img = image.load_img('./train/' + train + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    train_i.append(x)
    

In [63]:
base_model = ResNet101(weights = 'imagenet', include_top = True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
model.summary()

179650560/179648224 [==============================] - 50s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

In [ ]:
train_preds = []
validate_preds = []

for i in tqdm(train_i):
    pred = model.predict(i)
    train_preds.append(pred)

for i in tqdm(validate_i):
    pred = model.predict(i)
    validate_preds.append(pred)



100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [06:18<00:00,  3.17it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Anaconda\envs\CV\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\USER\AppData\Local\Temp/ipykernel_13104/1469841949.py", line 13, in <module>
    pickle.dump(validate_preds,f)
NameError: name 'pickle' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\CV\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\CV\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "D:\Anaconda\envs\CV\lib\sit

In [71]:
f = open(f'./validate_resnet101.pckl','wb')
pickle.dump(validate_preds,f)
f.close()

f = open(f'./validate_train_resnet101.pckl','wb')
pickle.dump(train_preds,f)
f.close()

In [88]:
validate_train_xy = pd.read_csv('validate_train.csv')
validate_xy =  pd.read_csv('validate.csv')
validate_path = validate_xy['id'].values
train_path = validate_train_xy['id'].values

In [86]:
best_train = []
for validate in tqdm(validate_preds):
    match = [np.sum((train-validate)**2)**0.5 for train in train_preds]
    best_match = np.argsort(match)#[:3]
    row = validate_train_xy.iloc[best_match]['id'].values
    best_train.append(row)

df = pd.DataFrame(best_train, columns=[f'{i+1}' for i in range(6300)])
df.to_csv('validate_resnet101_matches.csv',index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [01:47<00:00, 11.20it/s]


In [89]:
out=[]
for i in range(len(df)):
    coor = [validate_train_xy[validate_train_xy['id']== df[label][i]] for label in ['1','2','3']]
    result = pd.concat(coor)
    out.append([validate_path[i], np.mean(result['x']), np.mean(result['y'])])
    
out_csv = pd.DataFrame(out, columns=['id','x','y'])
out_csv.to_csv('validate_resnet101v2_top3_out.csv',index=False)

In [91]:
resnet101_predict = pd.read_csv('validate_resnet101v2_top3_out.csv')

In [104]:
MAE = np.abs(resnet101_predict ['x']-validate_xy['x']) +  np.abs(resnet101_predict ['y']-validate_xy['y'])
MAE = np.sum(MAE)/1200

In [105]:
MAE

19.462861110991113